# From the RBM to the Generalized Ising Model


## Introduction

Let us start from the energy function of a trained Restricted Boltzmann Machine (RBM):

$$
 \mathcal{H}(\boldsymbol{v}, \boldsymbol{h}) = - \sum_{j=1}^{N_v} b_j v_j - \sum_{i=1}^{N_h} c_i h_i - \sum_{j=1}^{N_v} \sum_{i=1}^{N_h}  h_i W_{ij} v_j , \tag{1}
$$

with binary variables $v_j, h_i \in \{0, 1 \}$. By applying a change of variables and then marginalizing over the hidden nodes, one can obtain an expression for the energy solely dependent upon the visible nodes $s_j \in \{ -1, 1 \}$:

$$
\begin{align}
\mathcal{H}(\boldsymbol{\sigma}) 
&= - \sum_j \left( b_j + \frac{1}{2} \sum_i W_{ij} \right) \sigma_j - \sum_i \ln \cosh \left[ \frac{1}{4} \sum_j W_{ij} \sigma_j + \frac{1}{2} \left( c_i + \frac{1}{2} \sum_j W_{ij} \right) \right] \\ 
&= - \sum_j \eta_j \sigma_j - \sum_i \ln \cosh \left( \sum_j w_{ij} \sigma_j + \theta_i \right), \tag{2}
\end{align}
$$

where the following variables were defined

$$
\eta_j \equiv \frac{1}{2} \left( b_j + \frac{1}{2} \sum_i W_{ij} \right), \ \theta_i \equiv \frac{1}{2} \left( c_i + \frac{1}{2} \sum_j W_{ij} \right), \ w_{ij} \equiv \frac{1}{4} W_{ij}. \tag{3}
$$

It can be we showed that the effective Hamiltonian given in the Eq. $(2)$ can be expanded as a generalized Ising Hamiltonian of the form
 
$$ \begin{align} 
\mathcal{H}_{\mathrm{GIM}}(\boldsymbol{\sigma}) 
&= - \sum_j H_j \sigma_j - \sum_{j_1 > j_2} J^{(2)}_{j_1 j_2} \sigma_{j_1} \sigma_{j_2} - \sum_{j_1 > j_2 > j_3} J_{j_1 j_2 j_3}^{(3)} \sigma_{j_1} \sigma_{j_2} \sigma_{j_3} + \dots \nonumber + \sum_{j_1 > \cdots > j_k} J^{(n)}_{j_1,\ldots,j_n}\sigma_{j_1} \cdots \sigma_{j_n} + \dots,
\tag{4}
\end{align}$$
 
The exact derivation of the mapping beetween the RBM parameters $\eta_j, \theta_i, w_{ij}$ and the couplings $H_{j_1 \dots j_n}$ is provided in [1]. Here, we will numerically implement the expressions provided in [1].

In [1]:
#importing libraries
import numpy as np
import matplotlib.pyplot as plt
import h5py

# numerical methods
from scipy.stats import norm
from scipy.integrate import simps

In [2]:
# use LaTeX fonts in the plot
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

## Loading the parameters of a saved RBM training

First, we need the parameters of an RBM trained with samples from the Ising model. Since this notebook will be exclusively dedicated to the numerical implementation of the mapping provided in [1], we will not delve into the generation of Ising model samples or the training of the models, which are described in a general manner also in [1].

In this context, we already have machines that have been trained with configurations of the the Ising model, which will serve as examples for introducing our methods. The files containing the parameters of these machines can be found in this repository under in `./model/(...)`.

In [ ]:
# Load RBM

f = h5py.File(f'./models/RBM_{training}.h5','r')
ep = 8000

# Initialize the variables
W = np.array(f['W'+str(ep)]) # dim: Nh x Nv
b = np.array(f['vbias'+str(ep)]) # dim: Nv
c = np.array(f['hbias'+str(ep)]) # dim: Nh

# change of variables
w = 0.25*W # dim: Nh x Nv
eta = 0.5*b + w.sum(axis = 0) # dim: Nv
theta = 0.5*c +  w.sum(axis = 1) # dim: Nh

f.close()

## Reference

[1] Decelle, A., Furtlehner, C., Navas Gómez, A. J., & Seoane, B. (2023). Inferring effective couplings with Restricted Boltzmann Machines. arXiv preprint arXiv:2309.02292.